In [1]:
import os

In [2]:
os.chdir('../..')

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
import dgl
from ogb.nodeproppred import DglNodePropPredDataset, Evaluator
import faiss

from dgl.nn.pytorch.conv import SAGEConv

Using backend: pytorch


In [4]:
dataset = DglNodePropPredDataset(name='ogbn-arxiv', 
                                 root='data/dataset_dgl/')
dataset

DglNodePropPredDataset(1)

In [5]:
torch.cuda.is_available()

True

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [7]:
graph = dataset[0][0]
graph

Graph(num_nodes=169343, num_edges=1166243,
      ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [8]:
torch.mean(graph.out_degrees() + graph.in_degrees() * 1.)

tensor(13.7737)

## Build Graph from graph sage embedding distances

In [9]:
output_emb_file = 'models/graphsage_link_pred/full_graphsage_linkpred_39_h.npy'
max_k = 100
min_k = 1
min_cossim = 0.7

In [10]:
embeddings = np.load(output_emb_file)

In [11]:
emb_norm = embeddings / np.linalg.norm(embeddings, axis=1)[:, None]

In [12]:
index_cos = faiss.IndexFlatIP(emb_norm.shape[1])

In [13]:
index_cos.train(emb_norm)
index_cos.add(emb_norm)

In [14]:
index_cos.ntotal

169343

In [15]:
distances, indices = index_cos.search(emb_norm, max_k + 1)

In [16]:
mask = (distances > min_cossim)[:, 1:]
mask[:, :min_k] = True

In [17]:
mask.sum(axis=1)

array([100,  18, 100, ...,  89, 100,  63])

In [18]:
mask.sum(axis=1).min()

1

In [19]:
u = np.repeat(indices[:, 0], mask.sum(axis=1))
print(u.shape)
u

(8189268,)


array([     0,      0,      0, ..., 169342, 169342, 169342])

In [20]:
v = indices[:,1:].reshape(-1)[mask.reshape(-1)]
print(v.shape)
v

(8189268,)


array([ 12093,  36054,  30038, ..., 140170,  65475, 151105])

In [21]:
graph.remove_edges(graph.edge_ids(graph.edges()[0], graph.edges()[1]))

In [22]:
graph.edges()

(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))

In [23]:
graph.add_edges(u, v)

In [24]:
graph.edges()

(tensor([     0,      0,      0,  ..., 169342, 169342, 169342]),
 tensor([ 12093,  36054,  30038,  ..., 140170,  65475, 151105]))

## Model

In [25]:
split_idx = dataset.get_idx_split()

In [26]:
torch.cuda.set_device(device)

In [27]:
features = graph.ndata['feat'].cuda()
labels = dataset.labels.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [28]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

<ipython-input-28-fa10932ea454>:1: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  train_nid = train_mask.nonzero().squeeze()


In [29]:
n_edges = graph.number_of_edges()

In [30]:
graph = graph.int().to(device)

In [31]:
n_iters = 3000
epochs = 1000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001

In [32]:
class ThreeLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim, dropout):
        super(ThreeLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(SAGEConv(input_dim, hidden_channels, 'mean'))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, hidden_channels, 'mean'))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, output_dim, 'mean'))
        
        self.dropout = dropout
        

    def forward(self, graph, x):
        x = self.convs[0](graph, x)
        x = self.bns[0](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[1](graph, x)
        x = self.bns[1](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[2](graph, x)
        return x.log_softmax(dim=-1)

In [33]:
model = ThreeLayer(input_dim,
                   hidden_channels,
                   output_dim,
                   0.5).cuda()

In [34]:
def train(model, graph, features, train_mask, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(graph, features)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [35]:
@torch.no_grad()
def test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator):
    model.eval()

    out = model(graph, features)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [36]:
evaluator = Evaluator(name='ogbn-arxiv')

In [37]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, graph, features, train_mask, optimizer)
    result = test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.2505, Train: 68.66%, Valid: 68.44% Test: 66.99%
Epoch: 200, Loss: 1.0859, Train: 70.89%, Valid: 69.92% Test: 68.91%
Epoch: 300, Loss: 1.0072, Train: 72.15%, Valid: 70.54% Test: 69.57%
Epoch: 400, Loss: 0.9590, Train: 73.13%, Valid: 70.81% Test: 69.96%
Epoch: 500, Loss: 0.9181, Train: 73.93%, Valid: 71.13% Test: 70.27%
Epoch: 600, Loss: 0.8890, Train: 74.80%, Valid: 71.29% Test: 70.35%
Epoch: 700, Loss: 0.8632, Train: 75.53%, Valid: 71.46% Test: 70.47%
Epoch: 800, Loss: 0.8351, Train: 76.27%, Valid: 71.54% Test: 70.51%
Epoch: 900, Loss: 0.8163, Train: 76.97%, Valid: 71.63% Test: 70.74%
Epoch: 1000, Loss: 0.7923, Train: 77.74%, Valid: 71.78% Test: 70.62%
